In [ ]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.1 MB/s eta 0:00:00


In [ ]:
#!pip install -U bitsandbytes

In [ ]:
#!pip install "numpy<2" --force-reinstall

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset
import re
import pandas as pd

In [ ]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
# Load orangesum_5k dataset
dataset = load_dataset("giuliadc/orangesum_filtered_new_spaces")

# Function to clean & deduplicate text
def preprocess_text(text):
    text = text.strip()  # Remove extra spaces
    text = re.sub(r"\s+", " ", text)  # Normalize spaces
    return text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

orangesum-space-train.json:   0%|          | 0.00/26.7M [00:00<?, ?B/s]

orangesum-space-validation.json:   0%|          | 0.00/2.05M [00:00<?, ?B/s]

orangesum-space-test.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7864 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/611 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/543 [00:00<?, ? examples/s]

In [ ]:
# Apply preprocessing (clean, deduplicate)
dataset = dataset.map(lambda x: {"text": preprocess_text(x["text"])})
#count data points that have length more than 2900 characters
dataset = dataset.filter(lambda x: len(x["text"]) < 2900)

print("✅ Preprocessing complete. Total articles:", len(dataset["train"]))

Map:   0%|          | 0/7864 [00:00<?, ? examples/s]

Map:   0%|          | 0/611 [00:00<?, ? examples/s]

Map:   0%|          | 0/543 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7864 [00:00<?, ? examples/s]

Filter:   0%|          | 0/611 [00:00<?, ? examples/s]

Filter:   0%|          | 0/543 [00:00<?, ? examples/s]

✅ Preprocessing complete. Total articles: 5065


In [ ]:
# Model name
model_name = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = {
    "load_in_4bit": True,  # Enable 4-bit quantization
    "bnb_4bit_compute_dtype": torch.float16,  # Ensure computation is done in float16 to avoid warnings
    "torch_dtype": torch.float16,  # Use float16 for computation
    "device_map": "auto",  # Automatically distribute model across GPUs if available
    "llm_int8_enable_fp32_cpu_offload": True,  # Enable CPU offloading for large models
}

# Load the model with the quantization configuration
model = AutoModelForCausalLM.from_pretrained(model_name, **bnb_config).eval()

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fix pad token issue
tokenizer.pad_token = tokenizer.eos_token

# Ensure the model is loaded
print("✅ Model loaded in 4-bit (bitsandbytes).")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

✅ Model loaded in 4-bit (bitsandbytes).


In [ ]:
model = model.to(device)

In [ ]:
# List of articles from dataset
articles = [item["text"] for item in dataset["train"]]

# Store results
new_summaries = []
no_summary_count = 0

In [ ]:
len(articles)

5065

In [ ]:
# Iterate through all articles
for i, article_text in enumerate(articles[2700:2800]):
    print(f"\nProcessing article {i+1}/{len(articles)}...")

    # Define few-shot prompt
    few_shot_prompt = """
    Voici des exemples d'articles et leurs résumés :

    Article : "Un réseau d'escrocs qui revendait des voitures de luxe achetées frauduleusement a été démantelé. L'opération a mobilisé 90 gendarmes en Gironde, dans le Var et les Pyrénées-Orientales. Dix suspects ont été arrêtés, avec un préjudice total de 2 millions d'euros."
    Résumé : "Dix personnes arrêtées après une enquête sur une escroquerie aux voitures de luxe, causant un préjudice de 2 millions d’euros."

    Article : "La saison 7 de Danse avec les stars s'est terminée, mais la saison 8 pourrait voir un changement majeur. Marie-Claude Pietragalla, juge depuis 2012, a indiqué qu'elle pourrait ne pas revenir en raison d'un emploi du temps chargé."
    Résumé : "Marie-Claude Pietragalla pourrait quitter Danse avec les stars avant la saison 8."

    Maintenant, résumez cet article en exactement 50 mots et placez le résumé entre les mots <debut> et </fin> :

    Article : "{}"
    Résumé :
    """.format(article_text)

    # Tokenize input
    inputs = tokenizer(
        few_shot_prompt,
        return_tensors="pt",
        truncation=True,
        padding="longest",
        max_length=2048  # Avoid cutting off longer articles
    )
    inputs = {key: (value.to(device).half() if key == "attention_mask" else value.to(device)) for key, value in inputs.items()}

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=150,  # Allow longer summaries
        num_beams=4,
        early_stopping=True
    )

    # Decode the generated summary
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(output)

    # Find the last occurrence of "Résumé :" and extract the next line
    match = re.findall(r'Résumé\s*:\s*<debut>(.*?)</fin>', output, re.DOTALL)

    if match:
        summary = match[-1]  # Get the last captured summary
    else:
        summary = "No summary found"
        no_summary_count += 1

    # Print the results
    print(f'Text: {article_text}')
    print(f'Summary: {summary}')

    # Store in list
    new_summaries.append({"text": article_text, "generated_summary": summary})

    # Print progress
    print(f"Summary {i+1} done.")

# Save to CSV
df = pd.DataFrame(new_summaries)
df.to_csv("generated_summaries_2700-2799.csv", index=False, encoding="utf-8")

print("\n All summaries saved to 'generated_summaries_2700-2799.csv'!")
print(f"No summary found for {no_summary_count} articles.")



Processing article 1/5065...

    Voici des exemples d'articles et leurs résumés :

    Article : "Un réseau d'escrocs qui revendait des voitures de luxe achetées frauduleusement a été démantelé. L'opération a mobilisé 90 gendarmes en Gironde, dans le Var et les Pyrénées-Orientales. Dix suspects ont été arrêtés, avec un préjudice total de 2 millions d'euros."
    Résumé : "Dix personnes arrêtées après une enquête sur une escroquerie aux voitures de luxe, causant un préjudice de 2 millions d’euros."

    Article : "La saison 7 de Danse avec les stars s'est terminée, mais la saison 8 pourrait voir un changement majeur. Marie-Claude Pietragalla, juge depuis 2012, a indiqué qu'elle pourrait ne pas revenir en raison d'un emploi du temps chargé."
    Résumé : "Marie-Claude Pietragalla pourrait quitter Danse avec les stars avant la saison 8."

    Maintenant, résumez cet article en exactement 50 mots et placez le résumé entre les mots <debut> et </fin> :

    Article : "Qui dit confinement, 